In [42]:
import pandas as pd
import numpy as np
import importlib 

df=pd.read_csv("./datasets/vehicles_merged.csv")

df.head()

,Manufacturer,Model,Fuel Type,Fuel Efficiency,EFF (km/l)/(km/kwh),Body Type,Seating Capacity,Ground Clearance (range),Body Type 1 (sedan),Body Type 2 (hatchback),...,Body Type 11 (kei / microvan),Body Type 12 (roadster),Body Type 13 (other),Body Type 14 (liftback),Body Type 15 (mpv / minivan),City/Urban,Suburban/Normal,Mid Off-Road,Off-Road/Hilly Terrain,Year
0,Audi,a1,Petrol,16-19,17.5,hatchback,5,121,0,1,...,0,0,0,0,0,1,0,0,0,2018–present
1,Audi,a1 25tfsi,Petrol,16-20,18.0,hatchback,5,136,0,1,...,0,0,0,0,0,1,1,0,0,2023-present
2,Audi,a1 25tfsi s-line,Petrol,16-20,18.0,hatchback,5,121,0,1,...,0,0,0,0,0,1,0,0,0,2018-present
3,Audi,a1 30tfsi,Petrol,17-19,18.0,hatchback,5,121,0,1,...,0,0,0,0,0,1,0,0,0,2018-present
4,Audi,a3,Petrol,17-22,19.5,hatchback,5,116,0,1,...,0,0,0,0,0,1,0,0,0,2012-2020


In [43]:
df=df.sort_values(by=["Manufacturer","Model","Year"],ascending=[True,True,True])
df=df.drop_duplicates(subset=["Manufacturer","Model","Year"],keep="first")





In [44]:
def convert_fuel_efficiency(row):
    fuel_type = str(row['Fuel Type']).lower()
    fe = row.get("Fuel Efficiency")

    if fe is None or str(fe).strip() == "":
        return float("nan")

    fe = str(fe).strip().lower()
    fe = fe.replace("km/h", "km/l")  # fix typos

    # Handle ranges like "16-20 km/l" or "4-5 km/kwh"
    if '-' in fe:
        parts = fe.replace('km/l','').replace('km/kwh','').split('-')
        try:
            low, high = map(float, parts)
            avg =round(( (low + high) / 2),1)
        except:
            return float("nan")  # fallback if parsing fails
    else:
        try:
            avg = float(fe.replace('km/l','').replace('km/kwh','').strip())
        except:
            return float("nan")

    

    # Electric vehicles → leave as-is (no conversion)
    # Petrol / Hybrid → leave as-is

    return avg

# Overwrite all values in 'Eff (km/l)'
df['EFF (km/l)/(km/kwh)'] = df.apply(convert_fuel_efficiency, axis=1)

desired_order = [
    "Manufacturer", "Model",  "Fuel Type","Fuel Efficiency","EFF (km/l)/(km/kwh)", "Body Type", "Seating Capacity",
    "Ground Clearance (range)", "Body Type 1 (sedan)", "Body Type 2 (hatchback)",
    "Body Type 3 (suv)", "Body Type 4 (mpv)", "Body Type 5 (pickup)", 
    "Body Type 6 (coupe)", "Body Type 7 (convertible)", "Body Type 8 (wagon)",
    "Body Type 9 (van)", "Body Type 10 (crossover)", "Body Type 11 (kei / microvan)",
    "Body Type 12 (roadster)", "Body Type 13 (other)", "Body Type 14 (liftback)",
    "Body Type 15 (mpv / minivan)", "City/Urban", "Suburban/Normal", 
    "Mid Off-Road", "Off-Road/Hilly Terrain","Year",
    
]

columns_to_use = [col for col in desired_order if col in df.columns]
df = df[columns_to_use]
df.to_csv("./datasets/vehicles_merged.csv", index=False)
